<a href="https://colab.research.google.com/github/B11GDEN/cvr_homework/blob/main/localization_hw8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Локализация. Упражнения
В этих упражнениях вам предлагается реализовать простые алгоритмы локализации роботов. На лекции, помимо фильтра частиц, были упомянуты методы триангуляции и альфа-бета фильтр. Давайте их реализуем: обе реализации должны быть сделаны в виде класса, у которого должен быть метод update. Метод update принимает на вход измеренения с камеры о положении ориентиров в сосбтвенной системе координат робота.

В качестве ориентиров будем использовать координаты стоек ворот. Для удобства будем пользоваться упрощенным вариантом, при котором ворота разных цветов. В словаре храняться положения стоек в глобальной системе координат (точка (0, 0) нахождится в центре поля)

In [1]:
landmarks = {
    "blue_posts": [
        [ -4.5, -1.0], [ -4.5, 1.0]
    ],
    "yellow_posts":[
        [ 4.5, -1.0], [ 4.5, 1.0]
    ]
}

Пример данных, которые могут поступать в модуль на одном шаге всей системы. Данные во втором словаре – это данные одометрии с собвственной информацией робота о перемещении за один шаг системы. Для проверки работы придумайте и запишите набор данных с такой же структурой.

In [2]:
observations = {"yellow_posts":[[4.45, -0.99],[4.49, 1.09]], "blue_posts":[]}
shift = {'shift_x':0.0, 'shift_y':0.01, 'shift_angle':-0.4}

## 1) Реализовать локализацию триангуляцией (3 балл)

In [3]:
class localization_triangulation:
    
    def __init__(self, x_start, y_start):
        '''
        Инициализируем начальное положение
        '''
        self.x = x_start
        self.y = y_start
    
    def update(self, distances):
        '''
        distances - словарь с расстояниями до стоек ворот
        Пример, distances = {"yellow_posts":[1.1, 0.9], "blue_posts":[]}
        '''

        #видны желтые стойки ворот
        if distances["blue_posts"] == []:
            r1 = distances["yellow_posts"][0]
            r2 = distances["yellow_posts"][1]
            l = landmarks["yellow_posts"][1][1] - landmarks["yellow_posts"][0][1]
            p = (r1 + r2 + l) / 2
            s = (p*(p-r1)*(p-r2)*(p-l))**(1/2)
            h = 2*s/l
            self.y = landmarks["yellow_posts"][0][0] - h
            if r2**2 > l**2 + r1**2:
                l1 = (r1**2 - h**2)**(1/2)
                self.x = landmarks["yellow_posts"][0][1] - l1
            if r1**2 > l**2 + r2**2:
                l2 = (r2**2 - h**2)**(1/2)
                self.x = landmarks["yellow_posts"][1][1] + l2
            else:
                l1 = (r1**2 - h**2)**(1/2)
                self.x = landmarks["yellow_posts"][0][1] + l1
        #видны синие стойки ворот
        else:
            r1 = distances["blue_posts"][0]
            r2 = distances["blue_posts"][1]
            l = landmarks["blue_posts"][1][1] - landmarks["blue_posts"][0][1]
            p = (r1 + r2 + l) / 2
            s = (p*(p-r1)*(p-r2)*(p-l))**(1/2)
            h = 2*s/l
            self.y = landmarks["blue_posts"][0][0] + h
            if r2**2 > l**2 + r1**2:
                l1 = (r1**2 - h**2)**(1/2)
                self.x = landmarks["blue_posts"][0][1] - l1
            if r1**2 > l**2 + r2**2:
                l2 = (r2**2 - h**2)**(1/2)
                self.x = landmarks["blue_posts"][1][1] + l2
            else:
                l1 = (r1**2 - h**2)**(1/2)
                self.x = landmarks["blue_posts"][0][1] + l1

### Проверка

In [8]:
# Преобразует позицию робота в расстояния до ворот
def positions2distances(positions):
    dst_y1 = ((positions["y"] - landmarks["yellow_posts"][0][0])**2 + (positions["x"] - landmarks["yellow_posts"][0][1])**2)**(1/2)
    dst_y2 = ((positions["y"] - landmarks["yellow_posts"][1][0])**2 + (positions["x"] - landmarks["yellow_posts"][1][1])**2)**(1/2)

    dst_b1 = ((positions["y"] - landmarks["blue_posts"][0][0])**2 + (positions["x"] - landmarks["blue_posts"][0][1])**2)**(1/2)
    dst_b2 = ((positions["y"] - landmarks["blue_posts"][1][0])**2 + (positions["x"] - landmarks["blue_posts"][1][1])**2)**(1/2)
    
    return {"yellow_posts": [dst_y1, dst_y2], "blue_posts": [dst_b1, dst_b2]}

In [7]:
Robot = localization_triangulation(0, 0)

positions1 = {"x": 1, "y": 1}
distances1 = positions2distances(positions1)
distances1["yellow_posts"] = []
Robot.update(distances1)
print(Robot.x, Robot.y)

positions2 = {"x": 2, "y": 2}
distances2 = positions2distances(positions2)
distances2["blue_posts"] = []
Robot.update(distances2)
print(Robot.x, Robot.y)

positions3 = {"x": 1, "y": 5}
distances3 = positions2distances(positions3)
distances3["yellow_posts"] = []
Robot.update(distances3)
print(Robot.x, Robot.y)

1.0 1.0
1.9999999999999973 1.9999999999999991
0.9999999999999574 5.000000000000009


## 2) Дополнить предыдущий пункт до альфа-бета фильтра (5 балла)

In [10]:
class localization_alphabeta:
    
    def __init__(self, x_start, y_start, alpha, beta):
        '''
        Инициализируем начальное положение
        '''
        self.x = x_start
        self.y = y_start

        self.local_tri =  localization_triangulation(x_start, y_start)

        self.alpha = alpha
        self.beta = beta
    
    def update(self, distances, shift):
        
        self.local_tri.update(distances)
        x_tri = self.local_tri.x
        y_tri = self.local_tri.y

        x_odo = self.x + shift["x"]
        y_odo = self.y + shift["y"]

        self.x = x_tri*self.alpha + x_odo*self.beta
        self.y = y_tri*self.alpha + y_odo*self.beta